In [1]:
import pandas as pd
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
eurostat = pd.read_csv('./data/eurostat/eurostat-2013.csv')

In [3]:
# rename columns to only use attributes indifiers
eurostat.rename(columns={ eurostat.columns[4]: 'teilmF', eurostat.columns[5]: 'teilmM' }, inplace=True)
eurostat.rename(columns=lambda s: re.sub('\(.*\)', '', s.split(' ', 1)[0]), inplace=True)

eurostat.describe()
eurostat.head()

,Nom,Code,tps00001,tec00115,teilmF,teilmM,tec00118,teimf050,tsdsc260,tet00002,tsc00001,tsc00004
0,Autriche,AT,8451860,0.3,5.3,4.9,2.1,2.17,4.1,-5683,2.84,38637
1,Belgique,BE,11161642,0.2,8.1,8.8,1.2,2.43,3.9,14145,2.24,44052
2,Bulgarie,BG,7284552,0.9,11.8,13.7,0.4,3.43,6.6,-3610,0.64,11295
3,Suisse,CH,8039060,1.9,4.6,4.1,0.1,0.90,4.4,18780,2.87,25142
4,Chypre,CY,865878,-5.4,15.5,17.5,0.4,6.00,4.9,-3229,0.46,895


In [4]:
def divide_by_population(row):
    population = row['tps00001']
    row['teilmF'] /= population
    row['teilmM'] /= population
    row['tsdsc260'] /= population
    row['tsc00004'] /= population

    return row

In [5]:
eurostat = eurostat.apply(divide_by_population, axis=1) # divide some rows by the population row value
eurostat = eurostat.drop(['tps00001'], axis=1) # delete the population column
eurostat.head()

,Nom,Code,tec00115,teilmF,teilmM,tec00118,teimf050,tsdsc260,tet00002,tsc00001,tsc00004
0,Autriche,AT,0.3,6.270809e-07,5.797540e-07,2.1,2.17,4.851003e-07,-5683,2.84,0.004571
1,Belgique,BE,0.2,7.256997e-07,7.884145e-07,1.2,2.43,3.494110e-07,14145,2.24,0.003947
2,Bulgarie,BG,0.9,1.619866e-06,1.880692e-06,0.4,3.43,9.060269e-07,-3610,0.64,0.001551
3,Suisse,CH,1.9,5.722062e-07,5.100099e-07,0.1,0.90,5.473277e-07,18780,2.87,0.003127
4,Chypre,CY,-5.4,1.790091e-05,2.021070e-05,0.4,6.00,5.658996e-06,-3229,0.46,0.001034


In [6]:
# apply a normalization filter : StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

numerical_columns = ['tec00115', 'teilmF', 'teilmM', 'tec00118', 'teimf050', 'tsdsc260', 'tet00002', 'tsc00001', 'tsc00004']
X_norm = pd.DataFrame(scaler.fit_transform(eurostat[numerical_columns]), columns=numerical_columns)
y = eurostat[['Nom', 'Code']]
X_norm.head()

,tec00115,teilmF,teilmM,tec00118,teimf050,tsdsc260,tet00002,tsc00001,tsc00004
0,-0.054714,-0.487096,-0.488163,0.882521,-0.663659,-0.381770,0.016166,1.108564,0.722569
1,-0.104869,-0.463321,-0.442513,-0.097691,-0.515720,-0.442140,0.205697,0.462228,0.322147
2,0.246214,-0.247752,-0.203546,-0.968991,0.053279,-0.194493,0.035982,-1.261334,-1.213797
3,0.747760,-0.500326,-0.503422,-1.295728,-1.386287,-0.354084,0.250002,1.140880,-0.202988
4,-2.913529,3.677346,3.806666,-0.968991,1.515605,1.920175,0.039624,-1.455235,-1.545133


In [7]:
# ACP
from sklearn.decomposition import PCA
acp = PCA(svd_solver='full')
coord = acp.fit_transform(X_norm)

n = X_norm.shape[0] # number of rows
p = X_norm.shape[1] # number of columns

In [8]:
# plot instances on the first plan (first 2 factors)
fig, axes = plt.subplots(figsize=(12,12))
axes.set_xlim(-4,9)
axes.set_ylim(-5,6)
for i in range(n):
    plt.annotate(y.values[i][1],(coord[i,0],coord[i,1]))
plt.plot([-4,9],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-5,6],color='silver',linestyle='-',linewidth=1)
plt.savefig('fig/acp_instances_1st_plan_CP1_CP2')
plt.close(fig)

# plot instances on the first plan (first 2 factors)
fig, axes = plt.subplots(figsize=(12,12))
axes.set_xlim(-4,9)
axes.set_ylim(-5,6)
for i in range(n):
    plt.annotate(y.values[i][1],(coord[i,2],coord[i,3]))
plt.plot([-4,9],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-5,6],color='silver',linestyle='-',linewidth=1)
plt.savefig('fig/acp_instances_1st_plan_CP3_CP4')
plt.close(fig)

In [9]:
# plot eigen values
eigval = float(n-1)/n*acp.explained_variance_
fig = plt.figure()
plt.plot(np.arange(1,p+1),eigval)
plt.title("Scree plot")
plt.ylabel("Eigen values")
plt.xlabel("Factor number")
plt.savefig('fig/acp_eigen_values')
plt.close(fig)

# print correlations between factors and original variables
sqrt_eigval = np.sqrt(eigval)
corvar = np.zeros((p,p))
for k in range(p):
    corvar[:,k] = acp.components_[k,:] * sqrt_eigval[k]
print(corvar)
# lines: variables
# columns: factors

[[-1.76468583e-01 -4.19442008e-01  7.48402361e-01  1.23878534e-01
   4.05404583e-01 -2.11006273e-01  7.99116972e-02 -4.70341316e-02
   8.02459045e-04]
 [ 9.03254621e-01 -3.82632520e-01 -1.54792486e-01  2.41756322e-02
  -7.58321262e-02  2.65116789e-02  3.93169440e-02 -6.49501951e-02
   3.09925507e-02]
 [ 9.03057280e-01 -3.83350814e-01 -1.38685254e-01  2.04170639e-02
  -5.86182393e-02  5.81415552e-02  3.94970744e-02 -9.34203331e-02
  -2.78447384e-02]
 [-1.97912234e-01 -2.84568677e-01  4.96657408e-01 -3.83782206e-01
  -6.95549701e-01 -4.37662683e-02  1.35205319e-02 -2.50626598e-03
  -3.05512709e-04]
 [ 4.75833503e-01  7.29586928e-01 -1.08882470e-01  1.95597245e-01
  -2.06051472e-01 -3.62556014e-01  1.31354188e-01  3.43943180e-03
  -1.37571483e-03]
 [ 7.99611212e-01 -5.67421225e-01  2.35248551e-02  9.95341653e-03
   5.09790018e-02 -9.35022841e-02 -1.26633359e-02  1.62754627e-01
  -3.40722533e-03]
 [ 7.92828040e-02  7.21485571e-02 -2.36924986e-01 -9.13510877e-01
   2.81099807e-01 -1.3460861

In [10]:
# draw correlation circles
from tp2_1_1 import correlation_circle

# CP1 & CP2
correlation_circle(df=X_norm, nb_var=p, x_axis=0, y_axis=1, corvar=corvar, plt=plt)
# CP3 & CP4
correlation_circle(df=X_norm, nb_var=p, x_axis=2, y_axis=3, corvar=corvar, plt=plt)

In [11]:
# Compute opptimal K value
from sklearn.cluster import KMeans

# Compute R-square, i.e. V_inter/V
from R_square_clustering import r_square
from purity import purity_score

# Plot elbow graphs for KMeans using R square and purity scores
lst_k=range(2,9)
lst_rsq = []
for k in lst_k:
    est=KMeans(n_clusters=k)
    est.fit(X_norm)
    lst_rsq.append(r_square(X_norm.to_numpy(), est.cluster_centers_,est.labels_,k))
    
fig = plt.figure()
plt.plot(lst_k, lst_rsq, 'bx-')
plt.xlabel('k')
plt.ylabel('RSQ')
plt.title('The Elbow Method showing the optimal k')
plt.savefig('fig/k-means_elbow_method')
plt.close()

In [12]:
# Clusters centroids
from mpl_toolkits.mplot3d import Axes3D
n_clusters = 3
est = KMeans(n_clusters=n_clusters)
title = '3 clusters'
est.fit(X_norm)
labels = est.labels_

# print centroids associated with several countries
lst_countries=['FR', 'JP']
# centroid of the entire dataset
# est: KMeans model fit to the dataset
# print (est.cluster_centers_)
for name in lst_countries:
    num_cluster = est.labels_[y.loc[y['Code']==name].index][0]
    print('Num cluster for '+name+': '+str(num_cluster))
    print('\tlist of countries: '+', '.join(y['Nom'].iloc[np.where(est.labels_==num_cluster)].values))
    print('\tcentroid: '+str(est.cluster_centers_[num_cluster]))

Num cluster for FR: 1
	list of countries: Autriche, Belgique, Suisse, République tchèque, Allemagne, Danemark, Zone euro, Estonie, Union européenne, Finlande, France, Japon, Pays-Bas, Suède, Slovénie, Royaume-Uni, États-Unis
	centroid: [ 0.03379404 -0.37950641 -0.38241934  0.23289022 -0.61412286 -0.33604868
 -0.06402644  0.84115809  0.58217842]
Num cluster for JP: 1
	list of countries: Autriche, Belgique, Suisse, République tchèque, Allemagne, Danemark, Zone euro, Estonie, Union européenne, Finlande, France, Japon, Pays-Bas, Suède, Slovénie, Royaume-Uni, États-Unis
	centroid: [ 0.03379404 -0.37950641 -0.38241934  0.23289022 -0.61412286 -0.33604868
 -0.06402644  0.84115809  0.58217842]


In [20]:
# Hierarchical clustering
from scipy.cluster.hierarchy import dendrogram, linkage

lst_labels = map(lambda pair: pair[0]+str(pair[1]), zip(y['Code'].values,y.index))
linkage_matrix = linkage(X_norm, 'ward')
fig = plt.figure()
dendrogram(
    linkage_matrix,
    color_threshold=7,
    labels=list(lst_labels),
)
plt.title('Hierarchical Clustering Dendrogram (Ward)')
plt.xlabel('sample index')
plt.ylabel('distance')
plt.tight_layout()
plt.savefig('fig/hierarchical-clustering')
plt.close()